# Texturing Functions

## Modify obj file to accept materials, create mtl files

In [92]:
def stripMaterialInformation(lines): #strip any preexisting material information
    result = []
    for line in lines:
        if(len(line)>= 6 and line[0:6] in ["mtllib", "usemtl"]):
            pass
        else:
            result.append(line)
    return result

In [93]:
LOCATION = "./"
OUT_LOCATION = "./Textured/"
FILENAME = "ReflectionTestWIthCube"
MTL_FILE_NAME = None or FILENAME + "_MTL"
newFile = "" #string with content of modified file
lines = None;
with open(LOCATION + FILENAME + ".obj", "r") as f:
    lines = f.readlines()


lines = stripMaterialInformation(lines)

lines.insert(0, f"mtllib {MTL_FILE_NAME}.mtl\n")


objects = []
curObject = None
doJoin = True
for line in lines:
    if(line[0] == "o"):
        curObject = line.split(" ")[1]
        objects.append(curObject.strip())
        doJoin = True
    elif(line[0] == "s"):
        newFile += line
        newFile += f"usemtl mat_{curObject}\n"
        doJoin = False
    elif(len(line) >= 6 and line[0:6] == "usemtl"):
        doJoin = False
    if(doJoin):
        newFile += line
    else:
        doJoin = True
print(newFile)

mtllib ReflectionTestWIthCube_MTL.mtl
# Blender v3.2.0 OBJ File: 'untitled.blend'
# www.blender.org
o Cube_Cube.021
v 0.639597 -1.000000 4.955940
v 0.639597 1.000000 4.955940
v -0.767028 -1.000000 3.534179
v -0.767028 1.000000 3.534179
v 2.061358 -1.000000 3.549315
v 2.061358 1.000000 3.549315
v 0.654733 -1.000000 2.127553
v 0.654733 1.000000 2.127553
vt 0.625000 0.000000
vt 0.375000 0.250000
vt 0.375000 0.000000
vt 0.625000 0.250000
vt 0.375000 0.500000
vt 0.625000 0.500000
vt 0.375000 0.750000
vt 0.625000 0.750000
vt 0.375000 1.000000
vt 0.125000 0.750000
vt 0.125000 0.500000
vt 0.875000 0.500000
vt 0.625000 1.000000
vt 0.875000 0.750000
vn -0.7109 0.0000 0.7033
vn -0.7033 0.0000 -0.7109
vn 0.7109 0.0000 -0.7033
vn 0.7033 0.0000 0.7109
vn 0.0000 -1.0000 0.0000
vn 0.0000 1.0000 0.0000
s off
usemtl mat_Cube_Cube.021

f 2/1/1 3/2/1 1/3/1
f 4/4/2 7/5/2 3/2/2
f 8/6/3 5/7/3 7/5/3
f 6/8/4 1/9/4 5/7/4
f 7/5/5 1/10/5 3/11/5
f 4/12/6 6/8/6 8/6/6
f 2/1/1 4/4/1 3/2/1
f 4/4/2 8/6/2 7/5/2
f 8/6/3 

In [94]:
# Make new mtl file

def getMatLines(mtl_name, location):
    return [f"\nnewmtl mat_{mtl_name}",
    "Ns 250.000000",
    "Ka 1.000000 1.000000 1.000000",
    "Kd 0.097925 0.065680 0.800000",
    "Ks 0.500000 0.500000 0.500000",
    "Ke 0.000000 0.000000 0.000000",
    "Ni 1.450000",
    "d 1.000000",
    "illum 2",
    f"map_Kd ./images/{location}.png"]


def writeNewMtl():
    with open(OUT_LOCATION + MTL_FILE_NAME + ".mtl", "w+") as f:
        for object in objects:
            f.writelines(line + "\n" for line in getMatLines(object, f"mat_{object}"))

def writeNewObj():
    with open(OUT_LOCATION +  FILENAME + "_textured.obj", "w+") as f:
        f.write(newFile)


writeNewMtl()
writeNewObj()

## File and data parser

In [95]:
from collections import defaultdict
import os
class hit:
    def __init__(self, name, hit_num, coords) -> None:
        self.name = name
        self.hit_num = hit_num
        self.x, self.y = coords
    def __repr__(self) -> str:
        return f"{self.name} - {self.hit_num}: <{self.x}, {self.y}>"


def parser():
    lines = 0
    hits = defaultdict(list)

    for filename in os.listdir("./output"):
        print(f"Parsing {filename}")
        with open("./output/output_0.txt", "r") as f:
            lines = f.readlines()
        for line in lines:
            line = line.strip()
            split = line.split("\t")
            name = split[0]
            hit_num = int(split[1])
            coords = [float(n) for n in split[2].split(",")]
            hits[name].append(hit(name, hit_num, coords))
        #print(fs)
    return hits
data = parser()
#data

Parsing output_0.txt
Parsing output_8.txt
Parsing output_1.txt
Parsing output_3.txt
Parsing output_15.txt
Parsing output_5.txt
Parsing output_7.txt
Parsing output_6.txt
Parsing output_2.txt
Parsing output_12.txt
Parsing output_13.txt
Parsing output_9.txt
Parsing output_4.txt
Parsing output_10.txt
Parsing output_11.txt
Parsing output_14.txt


## Turn data into image

In [96]:
from PIL import Image
import numpy as np


N = 200 #size of images
images = {}
for key in data:
    hits = data[key]
    image = np.zeros((N, N, 3), dtype=np.uint8)
    image.fill(255)

    for hit in hits:
        x = int(N*hit.x)
        y = int((N-1)-N*hit.y)
        if(hit.hit_num == 0):
            image[y][x] = [255,0,0]
        elif(hit.hit_num == 1):
            image[y][x] = [0,255,0]
    images[key] = Image.fromarray(image)

for key in images:
    images[key].save(f"./Textured/images/mat_{key}.png")
